In [1]:
#%%capture
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import json

def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st.strip()

def get_entities(query):
    query = clean(query)
    entities = []
    relations = []
    all_good = []
    words = query.split(" ")
    for word in words:
        if word.startswith("orkg"):
            all_good.append(word)

    for word in all_good:
        if word.startswith("orkgp:"):
            relations.append(word)
        else:
            entities.append(word)

    return {"entities": entities, "relations": relations}



sciqa_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["question"]
    outputs = examples["sparql_query"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = sciqa_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, Dataset
raw_dataset = load_dataset("orkg/SciQA", split = "train")

new_dataset = []
for sample in raw_dataset:
    ents = get_entities(sample["query"]["sparql"])
    sample["entities"] = ents["entities"]
    sample["relations"] = ents["relations"]
    row = {"question": sample["question"]["string"],
           "sparql_query": clean(sample["query"]["sparql"]),
           "instruction": "Given an English text describing an information need, generate a SPARQL query to retrieve relevant data from the SciQA Knowledge Graph using " + json.dumps(sample["entities"]) + " as entities and " + json.dumps(sample["relations"]) + " as relations"
           }
    new_dataset.append(row)

dataset = Dataset.from_list(new_dataset)
dataset = dataset.map(formatting_prompts_func, batched=True, )


README.md:   0%|          | 0.00/6.08k [00:00<?, ?B/s]

SciQA.py:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/199k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/62.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

Map:   0%|          | 0/1795 [00:00<?, ? examples/s]

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = None,
        num_train_epochs=10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 500,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/1795 [00:00<?, ? examples/s]

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/SCIQA

Mounted at /content/drive
/content/drive/MyDrive/SCIQA


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,795 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,240
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
500,0.121100
1000,0.053000
1500,0.043800
2000,0.038500


In [8]:
model.save_pretrained("Llama_3.1-8B_10_epochs_SCIQA_with_entities")
tokenizer.save_pretrained("Llama_3.1-8B_10_epochs_SCIQA_with_entities")

#model.save_pretrained_merged("Llama_3.1-8B_10_epochs_merged_4bit", tokenizer, save_method = "merged_4bit",)

# model.save_pretrained_gguf("Llama_3.1-8B_10_epochs_8bit.gguf", tokenizer,)

#model.save_pretrained_gguf("Llama_3.1-8B_10_epochs_q4_gguf", tokenizer, quantization_method = "q4_k_m")

('Llama_3.1-8B_10_epochs_SCIQA_with_entities/tokenizer_config.json',
 'Llama_3.1-8B_10_epochs_SCIQA_with_entities/special_tokens_map.json',
 'Llama_3.1-8B_10_epochs_SCIQA_with_entities/tokenizer.json')

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Llama_3.1-8B_10_epochs_DBLP_SCIQA", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
sciqa_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    sciqa_prompt.format(
        "Given an English text describing an information need, generate a SPARQL query to retrieve relevant data from the Open Research Knowledge Graph (ORKG)", # instruction
        "Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven an English text describing an information need, generate a SPARQL query to retrieve relevant data from the Open Research Knowledge Graph (ORKG)\n\n### Input:\nWhich model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?\n\n### Response:\nSELECT DISTINCT?model?model_lbl WHERE {?dataset a orkg:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "Story Cloze Test")?benchmark orkg:Benchmark; orkg:DATASET?dataset.?cont orkg:Cont; orkg:BENCHMARK?benchmark.?cont orkg:MODEL?model.?model rdfs:label?model_lbl. } ORDER BY DESC(?Accuracy) LIMIT 1<|end_of_text|>']